<img src=https://upload.wikimedia.org/wikipedia/commons/6/68/Logo_universidad_icesi.svg width=300>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sebastianb92/nlp-labs/blob/main/Session2/1-Word-Embeddings.ipynb)


# Maestría en Inteligencia Artificial  
## Procesamiento de Lenguaje natural
### Sesión 2 - Práctica

---


**Integrantes:**  
- Johan Sebastian Bonilla  
- Edwin Gómez  

# Introducción

# Clasificacion de textos - Analisis de reseñas y/u opiniones

En este notebook se aborda el problema de clasificación de texto aplicado al análisis de reseñas y opiniones en lenguaje natural. El objetivo principal es identificar automáticamente el sentimiento  presente en un conjunto de descripciones textuales utilizando técnicas de representación semántica.

# Configurar entorno

En esta sección se configuran las librerías y dependencias necesarias para el análisis de datos y procesamiento de lenguaje natural. Esto garantiza que el entorno esté listo para cargar, limpiar y analizar las conversaciones del chat político.

In [1]:
import sys
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

/tmp/ipython-input-273/3780340132.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/sebastianb92/nlp-labs/refs/heads/main/requirements.txt && pip install -r requirements.txt
    !python -m spacy download es_core_news_sm
else:
    !pip3 install -r ../requirements.txt

--2026-03-02 02:06:03--  https://raw.githubusercontent.com/sebastianb92/nlp-labs/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     282  --.-KB/s    in 0s      

2026-03-02 02:06:03 (8.29 MB/s) - ‘requirements.txt’ saved [282/282]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 29.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_cor

# 1. Recopilación de datos

Para el presente análisis se utilizará el conjunto de datos mteb/SpanishSentimentClassification, disponible en Hugging Face, el cual contiene reseñas y opiniones en español relacionadas con servicios de hospedaje. Cada registro del dataset se encuentra etiquetado según su polaridad de sentimiento, clasificándose en categorías positivas o negativas.

Este conjunto de datos permitirá evaluar modelos de representación semántica y clasificación automática en una tarea binaria de análisis de sentimiento aplicada a opiniones reales de usuarios.

In [14]:
from datasets import load_dataset
import warnings
import os

warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
dataset = load_dataset('sepidmnorozy/Spanish_sentiment', split='train')
# dataset = load_dataset("sentiment140", split="train")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'text'],
    num_rows: 1029
})

In [15]:
dataset[1]

{'label': 1,
 'text': 'un lugar hermosisimo , inolvidable. hermosa atención · rescato el servicio , como asi también la calidez de su gente y el hermoso entorno natural. nunca olvidaremos este viaje. rescato el lugar , la playa. la variedad de comida y la calidad. desde bien tempranito al cruzarte con el personal siempre hay un saludo y buena onda. el tamaño de las habitaciones es espectacular. nos gusto mucho el spa , es un lugar muy familiar y para descansar'}

In [20]:
for i in range(5):
    print(dataset[i])

{'label': 1, 'text': 'Está situado en el centro de la ciudad , con todo lo más turístico a tu alrededor ( El Pilar , por ejemplo ) , y sitios para tomar algo .'}
{'label': 1, 'text': 'un lugar hermosisimo , inolvidable. hermosa atención · rescato el servicio , como asi también la calidez de su gente y el hermoso entorno natural. nunca olvidaremos este viaje. rescato el lugar , la playa. la variedad de comida y la calidad. desde bien tempranito al cruzarte con el personal siempre hay un saludo y buena onda. el tamaño de las habitaciones es espectacular. nos gusto mucho el spa , es un lugar muy familiar y para descansar'}
{'label': 1, 'text': 'Todo absolutamente recomendable .'}
{'label': 1, 'text': 'El metro está a 50 metros bajando tan solo una calle .'}
{'label': 1, 'text': 'Y de Toscana ... mejor que lo veais ... y además delo bonito que es Florencia y Siena ... perderos por sus pueblos ..'}


In [21]:
from collections import Counter
Counter(dataset["label"])

Counter({1: 851, 0: 178})

In [22]:
import numpy as np
np.mean([len(t.split()) for t in dataset["text"]])

np.float64(16.289601554907676)

In [23]:
text_lengths = [len(row['text']) for row in dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 5
Texto más largo: 608
Longitud promedio: 84.85714285714286


In [25]:
import re
from collections import Counter

def simple_tokenizer(text):
    text = text.lower()
    text = re.sub(r"[^a-záéíóúüñ]+", " ", text)
    return text.strip().split()

# Construimos el vocabulario a partir de conjunto de datos.
token_counts = Counter()
for text in dataset["text"]:
    token_counts.update(simple_tokenizer(text))

# 50k-2 porque necesitamos reservar espacio para los dos tokens especiales
top_n_tokens = list(token_counts.keys())[:50000-2]
vocab = {"[PAD]": 0, "[UNK]": 1}
for token in top_n_tokens:
    vocab[token] = len(vocab)

def tokenize_text(text, max_length=50):
    tokens = simple_tokenizer(text)
    ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens[:max_length]]
    ids += [vocab["[PAD]"]] * (max_length - len(ids))
    return ids

# 2. Preprocesamiento de datos

En esta etapa se realiza la limpieza y preparación del texto con el fin de garantizar que la información pueda ser procesada adecuadamente por los modelos de aprendizaje automático.

El preprocesamiento incluye la eliminación de símbolos innecesarios, caracteres especiales y signos de puntuación que no aportan valor semántico al análisis. Asimismo, se convierten todos los textos a minúsculas para evitar duplicidad de términos causada por diferencias en capitalización.

Este proceso permite estandarizar el corpus, reducir ruido en los datos y mejorar la calidad de las representaciones vectoriales generadas posteriormente.

In [26]:
print(f"Vocabulario: {len(vocab)} tokens")
print("Primeros 15 tokens:")
print(f"{top_n_tokens[:15]}")
print("15 tokens de en medio:")
print(f"{top_n_tokens[1000:1015]}")
print("Últimos 15 tokens:")
print(f"{top_n_tokens[-15:]}")

Vocabulario: 2672 tokens
Primeros 15 tokens:
['está', 'situado', 'en', 'el', 'centro', 'de', 'la', 'ciudad', 'con', 'todo', 'lo', 'más', 'turístico', 'a', 'tu']
15 tokens de en medio:
['valía', 'dam', 'ana', 'frank', 'min', 'caminando', 'central', 'sale', 'categoria', 'reflejar', 'casos', 'francamente', 'defraudaron', 'estaciones', 'autobuses']
Últimos 15 tokens:
['mariscal', 'victorio', 'lucchino', 'veía', 'divertí', 'cafeteria', 'malas', 'cuidan', 'vi', 'mercure', 'costo', 'sabes', 'doonde', 'obstante', 'quizás']


## Resultados

Aunque la traducción mejora los resultados, persisten algunas limitaciones:

- **Pérdida de matices idiomáticos**: expresiones coloquiales colombianas pueden no traducirse bien.
- **Sarcasmo e ironía**: difíciles de preservar en traducción automática y aún con una buena traducción el sarcasmo es difícilmente interpretado por este modelo simple.
- **Contexto cultural**: referencias locales (RTVC, Petro) pueden perder significado.
- **Latencia**: la traducción añade tiempo de procesamiento.

### Hallazgos del Análisis

Los resultados con traducción revelan:
- **Mayor variabilidad** en los scores compound (ya no todo es neutral).
- **Detección de polarización**: se identifican mensajes claramente positivos y negativos, aunque no precisos debido al alto nivel de sarcasmo empleado en el chat.

### Conclusión

El enfoque de **traducción + VADER** es práctico en un entorno académico de análisis de sentimientos en español, pero pierde algunos matices culturales y no se recomienda esta aproximación en un entorno real. Se captura de mejor manera la polaridad general de los mensajes comparado con el tratamiento de los mensajes originales en español y permite identificar patrones de comunicación entre los participantes del chat político analizado.


@inproceedings{mollanorozy-etal-2023-cross,
  address = {Dubrovnik, Croatia},
  author = {Mollanorozy, Sepideh  and
Tanti, Marc  and
Nissim, Malvina},
  booktitle = {Proceedings of the 5th Workshop on Research in Computational Linguistic Typology and Multilingual NLP},
  doi = {10.18653/v1/2023.sigtyp-1.9},
  editor = {Beinborn, Lisa  and
Goswami, Koustava  and
Murado{\\u{g}}lu, Saliha  and
Sorokin, Alexey  and
Kumar, Ritesh  and
Shcherbakov, Andreas  and
Ponti, Edoardo M.  and
Cotterell, Ryan  and
Vylomova, Ekaterina},
  month = may,
  pages = {89--95},
  publisher = {Association for Computational Linguistics},
  title = {Cross-lingual Transfer Learning with \{P\}ersian},
  url = {https://aclanthology.org/2023.sigtyp-1.9},
  year = {2023},
}


@article{enevoldsen2025mmtebmassivemultilingualtext,
  title={MMTEB: Massive Multilingual Text Embedding Benchmark},
  author={Kenneth Enevoldsen and Isaac Chung and Imene Kerboua and Márton Kardos and Ashwin Mathur and David Stap and Jay Gala and Wissam Siblini and Dominik Krzemiński and Genta Indra Winata and Saba Sturua and Saiteja Utpala and Mathieu Ciancone and Marion Schaeffer and Gabriel Sequeira and Diganta Misra and Shreeya Dhakal and Jonathan Rystrøm and Roman Solomatin and Ömer Çağatan and Akash Kundu and Martin Bernstorff and Shitao Xiao and Akshita Sukhlecha and Bhavish Pahwa and Rafał Poświata and Kranthi Kiran GV and Shawon Ashraf and Daniel Auras and Björn Plüster and Jan Philipp Harries and Loïc Magne and Isabelle Mohr and Mariya Hendriksen and Dawei Zhu and Hippolyte Gisserot-Boukhlef and Tom Aarsen and Jan Kostkan and Konrad Wojtasik and Taemin Lee and Marek Šuppa and Crystina Zhang and Roberta Rocca and Mohammed Hamdy and Andrianos Michail and John Yang and Manuel Faysse and Aleksei Vatolin and Nandan Thakur and Manan Dey and Dipam Vasani and Pranjal Chitale and Simone Tedeschi and Nguyen Tai and Artem Snegirev and Michael Günther and Mengzhou Xia and Weijia Shi and Xing Han Lù and Jordan Clive and Gayatri Krishnakumar and Anna Maksimova and Silvan Wehrli and Maria Tikhonova and Henil Panchal and Aleksandr Abramov and Malte Ostendorff and Zheng Liu and Simon Clematide and Lester James Miranda and Alena Fenogenova and Guangyu Song and Ruqiya Bin Safi and Wen-Ding Li and Alessia Borghini and Federico Cassano and Hongjin Su and Jimmy Lin and Howard Yen and Lasse Hansen and Sara Hooker and Chenghao Xiao and Vaibhav Adlakha and Orion Weller and Siva Reddy and Niklas Muennighoff},
  publisher = {arXiv},
  journal={arXiv preprint arXiv:2502.13595},
  year={2025},
  url={https://arxiv.org/abs/2502.13595},
  doi = {10.48550/arXiv.2502.13595},
}

@article{muennighoff2022mteb,
  author = {Muennighoff, Niklas and Tazi, Nouamane and Magne, Loïc and Reimers, Nils},
  title = {MTEB: Massive Text Embedding Benchmark},
  publisher = {arXiv},
  journal={arXiv preprint arXiv:2210.07316},
  year = {2022}
  url = {https://arxiv.org/abs/2210.07316},
  doi = {10.48550/ARXIV.2210.07316},
}
